# 1. Read the corpus into the list of RSTTree structures

We need to get a list of trees to navigate the sentences and to later evaluate them

## Transform s-expression into nested lists

In [55]:
import re
  
def parse_sexp(sexp):
    """
    Transform lisp s-expression into nested list structure.
    All lisp features except for parentheses are ignored.

    Args:
        sexp: s-expression to transform.

    Returns:
        Nested lists representing s-expression.
    """        

    term_regex = r"""(?mx)
        \s*(?:
            (?P<lparen>\()|
            (?P<rparen>\))|
            (?P<s>(\w|\-)+)|
            (?P<p>[\.,]+)
           )"""
    stack = []
    out = []
    for termtypes in re.finditer(term_regex, sexp):
        term, value = [(t,v) for t,v in termtypes.groupdict().items() if v][0]
        if term == 'lparen':
            stack.append(out)
            out = []
        elif term == 'rparen':
            assert stack, "Trouble with nesting of parentheses"
            tmpout, out = out, stack.pop(-1)
            out.append(tmpout)
        elif term == 's' or term == 'p':
            out.append(value)
        else:
            raise NotImplementedError("Error: %r" % (term, value))
    assert not stack, "Trouble with nesting of parentheses"
    return out[0]

## Define tree class

In [56]:
import itertools
import numpy as np

class RSTTree:
    
    MONONUCLEAR = 0
    MULTINUCLEAR = 1
    
    def __init__(self):
        self.type = 'root'
        self.span = None
        self.text = None
        self.nuclei = []
        self.satellite = None
        self.is_leaf = False
        self.nuclearity = None
        self.index = -1
        self.relation = None        
        self.text_constructed = False
        self.flatten = lambda l: [item for sublist in l for item in sublist]
    
    @staticmethod
    def from_list(list_tree, filename, index = -1, parent = None):
        """
        Initialize RST tree from nested lists.

        Args:
            list_tree: nested lists represeting lisp trees.
            filename: name of the file corresponding to the lisp trees.
            index: index of the nuclues un multinuclear relations.
            parent: parent of this tree.

        Returns:
            A new RST tree.
        """

        new_tree = RSTTree()
        new_tree.parent = parent
        new_tree.filename = filename
        new_tree.nuclearity = RSTTree.MULTINUCLEAR
        new_tree.index = index
        
        nucleus_index = 0
        new_tree.type = list_tree[0].lower()
        for child in list_tree[1:]:
            child_tag = child[0]
            
            if child_tag == 'span':
                new_tree.span = int(child[1]), int(child[2])
            
            elif child_tag == 'leaf':
                new_tree.is_leaf = True
                new_tree.span = int(child[1]), int(child[1])
            
            elif child_tag == 'rel2par' and parent is not None and child[1] != "span":
                parent.relation = child[1]
            
            elif child_tag == 'Nucleus':
                new_tree.nuclei.append(RSTTree.from_list(child, filename, parent=new_tree, index=nucleus_index))
                nucleus_index += 1
            
            elif child_tag == 'Satellite':
                new_tree.satellite = RSTTree.from_list(child, filename, parent=new_tree)
                new_tree.nuclearity = RSTTree.MONONUCLEAR
            
            elif child_tag == 'text':
                new_tree.text = []
                for item in child[1:]:
                    if isinstance(item, list):
                        new_tree.text += item
                    else:
                        new_tree.text.append(item)

        return new_tree

    @staticmethod
    def from_text(text, start, end):
        """
        Initialize RST tree from text.

        Args:
            text: as a list of tokens.
            start: span starting index.
            end: span ending index.

        Returns:
            A new RST tree.
        """
        new_tree = RSTTree()
        new_tree.text = text
        new_tree.span = (start, end)
        new_tree.is_leaf = True
        new_tree.text_constructed = True
        return new_tree

    @staticmethod
    def from_trees(relation, nuclei, satellite=None):
        """
        Initialize RST tree from child trees.

        Args:
            relation: relation type for this tree.
            nuclei: list of child nuclei RST trees.
            staellite: child satellite RST tree.

        Returns:
            A new RST tree.
        """        
        new_tree = RSTTree()
        new_tree.is_leaf = False
        new_tree.nuclei = nuclei
        new_tree.satellite = satellite
        new_tree.relation = relation
        new_tree.nuclearity = RSTTree.MULTINUCLEAR if satellite is None else RSTTree.MONONUCLEAR
        new_tree.calculate_spans()
        
        for nucleus in new_tree.nuclei:
            nucleus.type = 'nucleus'
            nucleus.parent = new_tree
        
        if new_tree.satellite is not None:
            new_tree.satellite.type = 'satellite'
            new_tree.satellite.parent = new_tree
        
        return new_tree

    def output_lisp(self, indent = 0):
        """
        Output the tree in the s-expression format.

        Args:
            indent: indentation for the output.

        Returns:
            A pretty string containing s-expression.
        """        

        rep = (" " * indent) + "( " + self.type + " "
        # 1. leaf or span
        if self.is_leaf:
            rep += "(leaf " + str(self.span[0]) + ") "
        else:
            rep += "(span " + str(self.span[0]) + " " + str(self.span[1]) + ") "
        # 2. rel2par
        if self.type == "nucleus":
            if self.parent.nuclearity == RSTTree.MULTINUCLEAR:
                rep += "(rel2par " + self.parent.relation + ") "
            else:
                rep += "(rel2par span) "
        elif self.type == "satellite":
            rep += "(rel2par " + self.parent.relation + ") "
        # 3. children
        if self.is_leaf:
            rep += "(text " + " ".join(self.text) + ") )"
        else:
            for nucleus in self.nuclei:
                rep += (" " * indent) + "\n" + nucleus.output_lisp(indent + 2)
            if self.satellite is not None:
                rep += (" " * indent) + "\n" + self.satellite.output_lisp(indent + 2)
            rep += (" " * indent) + "\n)"
        return rep
    
    def calculate_spans(self):
        if self.span is None:
            start = np.inf
            end = 0
            if self.nuclei is not None:
                start = min([nucleus.calculate_spans()[0] for nucleus in self.nuclei])
                end = max([nucleus.calculate_spans()[1] for nucleus in self.nuclei])
            if self.satellite is not None:
                start = min(start, self.satellite.calculate_spans()[0])
                end = max(end, self.satellite.calculate_spans()[1])
            self.span = (start, end)
        return self.span
        
    def construct_text(self):
        self.text_constructed = True
        if self.text is None:
            texts = []
            if self.nuclei is not None:
                texts += [nucleus.construct_text() for nucleus in self.nuclei]
            if self.satellite is not None:
                texts.append(self.satellite.construct_text())
            self.text = list(itertools.chain.from_iterable(texts))
        return self.text

    def get_subtexts(self):
        assert self.text_constructed, "Texts are not constructed"
        subtexts = []
        if not self.is_leaf:
            if self.nuclei is not None:
                subtexts += [nucleus.text for nucleus in self.nuclei]
            if self.satellite is not None:
                subtexts.append(self.satellite.text)
        return subtexts
    
    def all_trees(self):
        subtrees = []
        if self.nuclei is not None:
            subtrees += [nucleus.all_trees() for nucleus in self.nuclei]
        if self.satellite is not None:
            subtrees.append(self.satellite.all_trees())
        subtrees = self.flatten(subtrees)
        if not self.is_leaf:
            subtrees.append(self)
        return subtrees

    def all_edus(self):
        if self.text is not None:
            return [self.text]
        else:
            edus = []
            edus += [nucleus.all_edus() for nucleus in self.nuclei]
            if self.satellite is not None:
                edus += self.satellite.all_edus()
            return edus

## Corpus reader

In [5]:
import os

class CorpusReader:

    def __init__(self, rst_root):
        self.rst_root = rst_root
    
    def load_test_trees(self):
        return self._load_trees("TEST")
    
    def load_train_trees(self):
        return self._load_trees("TRAINING")
    
    def _load_trees(self, dirsuffix):
        root_with_suffix = os.path.join(self.rst_root, dirsuffix)
        for dirname in os.listdir(root_with_suffix):
            dirname = os.path.join(root_with_suffix, dirname)
            if os.path.isdir(dirname):
                for filename in os.listdir(dirname):
                    filename = os.path.join(dirname, filename)
                    if os.path.isfile(filename) and len(filename) > 9 and filename[-9:] == "lisp.name":
                        with open(filename, encoding="utf-8") as file:
                            try:
                                contents = parse_sexp(file.read())
                            except AssertionError as err:
                                print(filename)
                                raise err
                            tree = RSTTree.from_list(contents, filename)
                            tree.construct_text()
                            yield tree

In [6]:
corpus_reader = CorpusReader("/Users/vpraid/Downloads/RSTDT/data/RSTtrees-WSJ-main-1.0")

# 2. Use gensim to transform sentences into word vectors

## Create EDU reader

In [7]:
class EduReader:
    
    def __init__(self, reader):
        self.reader = reader

    def __iter__(self):
        for tree in self.reader.load_train_trees():
            for edu in tree.all_edus():
                yield edu

edu_reader = EduReader(corpus_reader)

## Load gensim and create a model

In [10]:
import gensim
import logging
logging.basicConfig(level=logging.CRITICAL)

In [ ]:
EMBED_SIZE=100
embed_model = gensim.models.Word2Vec(edu_reader, size=EMBED_SIZE, min_count=2, window=5, iter=100)

In [11]:
def check_similar(model):
    pretrained_weights = model.wv.syn0
    vocab_size, emdedding_size = pretrained_weights.shape
    print('Result embedding shape:', pretrained_weights.shape)
    print('Checking similar words:')
    for word in ['money', 'bank', 'company']:
        most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in model.most_similar(word)[:4])
        print('  %s -> %s' % (word, most_similar))

check_similar(embed_model)

Result embedding shape: (7505, 100)
Checking similar words:
  money -> approach (0.42), choice (0.39), mega-issues (0.37), buying (0.36)
  bank -> RTC (0.43), Parisian (0.41), Weatherford (0.41), suspension (0.41)
  company -> transaction (0.42), firm (0.39), group (0.39), companies (0.38)


/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [12]:
def get_sentence_embedding(sentence):
        embeddings = [embed_model[word] for word in sentence if word in embed_model.wv.vocab]
        if len(embeddings) == 0:
            return None
        word_sum = np.zeros(EMBED_SIZE, dtype='float64')
        word_count = 0
        for word in embeddings:
            word_sum += word
            word_count += 1
        return word_sum / word_count

# 3. Get POS tags from spaCy

In [13]:
import spacy
from spacy.attrs import POS

nlp = spacy.load("en")

In [14]:
pos2idx = {}
def fill_pos_tags():
    for edu in edu_reader:
        doc = nlp(" ".join(edu))
        for token in doc:
            if token.pos_ not in pos2idx:
                pos2idx[token.pos_] = len(pos2idx)

fill_pos_tags()

In [15]:
def get_sentence_vector(sentence):
    embedding = get_sentence_embedding(sentence)
    if embedding is None:
        return None
    doc = nlp(" ".join(sentence))
    root = [token for token in doc if token.head == token][0]
    return np.r_[len(sentence), (np.arange(POS) == pos2idx[root.pos_]).astype(np.float64), embedding]

# 4. Train connection classifier

In [19]:
import tqdm
import pickle

from tqdm import tqdm_notebook, tnrange

tqdm.monitor_interval = 0

In [20]:
def are_connected(lhs, rhs):
    if lhs.parent != rhs.parent:
        return False
    if lhs.parent.nuclearity == RSTTree.MONONUCLEAR:
        return True
    assert lhs.type == 'nucleus' and rhs.type == 'nucleus'
    return np.abs(lhs.index - rhs.index) == 1

In [21]:
from itertools import product
from random import shuffle

def shuffled(x):
    y = x[:]
    shuffle(y)
    return y

def get_vector(lhs, rhs):
    if lhs.text is None or rhs.text is None:
        return None
    lhs_vector = get_sentence_vector(lhs.text)
    if lhs_vector is None:
        return None
    rhs_vector = get_sentence_vector(rhs.text)
    if rhs_vector is None:
        return None
    return np.r_[lhs_vector, rhs_vector]

def get_connection_set(trees):
    pairs = []
    labels = []
    for tree in tqdm_notebook(trees):
        
        subtrees = tree.all_trees()
        for subtree in subtrees:
            if subtree.nuclearity == RSTTree.MONONUCLEAR:
                pair = get_vector(subtree.nuclei[0], subtree.satellite)
                if pair is None:
                    continue
                pairs.append(pair)
                labels.append(1)
            else:
                for lhs, rhs in zip(subtree.nuclei, subtree.nuclei[1:]):
                    pair = get_vector(lhs, rhs)
                    if pair is None:
                        continue
                    pairs.append(pair)
                    labels.append(1)

        for i, (left, right) in enumerate(product(shuffled(subtrees), shuffled(subtrees))):
            if left == right or are_connected(left, right):
                continue
            if i > 30:
                break
            pair = get_vector(left, right)
            if pair is None:
                continue
            pairs.append(pair)
            labels.append(0)

    shape = len(pairs), pairs[0].shape[0]
    return np.concatenate(pairs).reshape(shape), np.array(labels)

## Prepare training connection data
Do not run this cell unless you know what you are doing. It takes 10 minutes to finish.

In [1029]:
conn_train_X, conn_train_Y = get_connection_set(corpus_reader.load_train_trees())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load training connection data

In [1030]:
with open('connection_train_set.pickle', 'wb') as handle:
    pickle.dump((conn_train_X, conn_train_Y), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
with open('connection_train_set.pickle', 'rb') as handle:
    (conn_train_X, conn_train_Y) = pickle.load(handle)

## Prepare test connection data

In [1050]:
conn_test_X, conn_test_Y = get_connection_set(corpus_reader.load_test_trees())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load test connection data

In [1161]:
with open('connection_test_set.pickle', 'wb') as handle:
    pickle.dump((conn_test_X, conn_test_Y), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
with open('connection_test_set.pickle', 'rb') as handle:
    (conn_test_X, conn_test_Y) = pickle.load(handle)

## Build model

In [25]:
import keras
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout

### Build and train the network

In [1116]:
connection_model = Sequential()
connection_model.add(Dense(256, input_dim=conn_train_X.shape[1], activation='relu'))
connection_model.add(Dropout(0.5))
connection_model.add(Dense(128, activation='relu'))
connection_model.add(Dropout(0.5))
connection_model.add(Dense(64, activation='relu'))
connection_model.add(Dropout(0.5))
connection_model.add(Dense(1, activation='sigmoid'))
connection_model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [1197]:
connection_model.fit(conn_train_X, conn_train_Y, batch_size=128, verbose=0, epochs=15)
connection_model.save("connection_model.hs")

### Or just load it

In [26]:
connection_model = load_model("connection_model.hs")

### And evaluate it

In [27]:
score = connection_model.evaluate(conn_train_X, conn_train_Y, verbose=0)
print(score)

[0.21345597905274691, 0.90359750197906585]


In [28]:
score = connection_model.evaluate(conn_test_X, conn_test_Y, verbose=0)
print(score)

[1.1005296130025364, 0.75411089853620439]


# 5. Train relation classifier

In [30]:
from keras.utils import to_categorical

def get_relation_set(trees, populate_relations = False, relation2idx = dict()):
    pairs = []
    labels = []
    for tree in tqdm_notebook(trees):
        
        subtrees = tree.all_trees()
        for subtree in subtrees:
            
            if subtree.relation is None:
                continue
            
            if populate_relations and subtree.relation not in relation2idx:
                relation2idx[subtree.relation] = len(relation2idx)
            
            elif not populate_relations and subtree.relation not in relation2idx.keys():
                continue
            
            if subtree.nuclearity == RSTTree.MONONUCLEAR:
                pair = get_vector(subtree.nuclei[0], subtree.satellite)
                if pair is None:
                    continue
                pairs.append(pair)
                labels.append(relation2idx[subtree.relation])
            
            else:
                for lhs, rhs in zip(subtree.nuclei, subtree.nuclei[1:]):
                    pair = get_vector(lhs, rhs)
                    if pair is None:
                        continue
                    pairs.append(pair)
                    labels.append(relation2idx[subtree.relation])

    shape = len(pairs), pairs[0].shape[0]
    return np.concatenate(pairs).reshape(shape), to_categorical(labels, num_classes=len(relation2idx)), relation2idx

## Prepare training relation data
Do not run this cell unless you know what you are doing. It takes 10 minutes to finish.

In [31]:
rel_train_X, rel_train_Y, relation2idx = get_relation_set(corpus_reader.load_train_trees(), True)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load training relation data

In [1183]:
with open('relation_train_set.pickle', 'wb') as handle:
    pickle.dump((rel_train_X, rel_train_Y, relation2idx), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
with open('relation_train_set.pickle', 'rb') as handle:
    (rel_train_X, rel_train_Y, relation2idx) = pickle.load(handle)

## Prepare test relation data

In [32]:
rel_test_X, rel_test_Y, _ = get_relation_set(
    corpus_reader.load_test_trees(),
    populate_relations=False,
    relation2idx=relation2idx)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load test relation data

In [1200]:
with open('relation_test_set.pickle', 'wb') as handle:
    pickle.dump((rel_test_X, rel_test_Y), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
with open('relation_test_set.pickle', 'rb') as handle:
    (rel_test_X, rel_test_Y) = pickle.load(handle)

## Build model

### Build and train the network

In [1201]:
relation_model = Sequential()
relation_model.add(Dense(256, input_dim=rel_train_X.shape[1], activation='relu'))
relation_model.add(Dropout(0.5))
relation_model.add(Dense(128, activation='relu'))
relation_model.add(Dropout(0.5))
relation_model.add(Dense(64, activation='relu'))
relation_model.add(Dropout(0.5))
relation_model.add(Dense(rel_train_Y.shape[1], activation='softmax'))
relation_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [1202]:
relation_model.fit(rel_train_X, rel_train_Y, batch_size=128, verbose=0, epochs=50)
relation_model.save("relation_model.h5")

### Or just load it

In [39]:
relation_model = load_model("relation_model.h5")

### And evaluate it

In [40]:
score = relation_model.evaluate(rel_train_X, rel_train_Y, verbose=0)
print(score)

[1.785708257890628, 0.53254663115179401]


In [41]:
score = relation_model.evaluate(rel_test_X, rel_test_Y, verbose=0)
print(score)

[2.5126443585398879, 0.41736334399395053]


# 6. Train nuclearity classifier

In [44]:
NUCLEUS_L = [1, 0]
NUCLEUS_R = [0, 1]
NUCLEUS_B = [1, 1]

def get_nuclearity_set(trees):
    pairs = []
    labels = []
    for tree in tqdm_notebook(trees):
        
        subtrees = tree.all_trees()
        for subtree in subtrees:
            
            if subtree.nuclearity == RSTTree.MONONUCLEAR:
                pair = get_vector(subtree.nuclei[0], subtree.satellite)
                if pair is None:
                    continue
                pairs.append(pair)
                label = NUCLEUS_L if subtree.nuclei[0].span[1] <= subtree.satellite.span[0] else NUCLEUS_R
                labels.append(label)
            
            else:
                for lhs, rhs in zip(subtree.nuclei, subtree.nuclei[1:]):
                    pair = get_vector(lhs, rhs)
                    if pair is None:
                        continue
                    pairs.append(pair)
                    labels.append(NUCLEUS_B)

    shape = len(pairs), pairs[0].shape[0]
    return np.concatenate(pairs).reshape(shape), np.array(labels)

## Prepare nuclearity relation data
Do not run this cell unless you know what you are doing. It takes 10 minutes to finish.

In [1232]:
nuc_train_X, nuc_train_Y = get_nuclearity_set(corpus_reader.load_train_trees())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load nuclearity training data

In [1236]:
with open('nuclearity_train_set.pickle', 'wb') as handle:
    pickle.dump((nuc_train_X, nuc_train_Y), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
with open('nuclearity_train_set.pickle', 'rb') as handle:
    (nuc_train_X, nuc_train_Y) = pickle.load(handle)

## Prepare nuclearity test data

In [1237]:
nuc_test_X, nuc_test_Y = get_nuclearity_set(corpus_reader.load_test_trees())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### Save / load nuclearity test data

In [1238]:
with open('nuclearity_test_set.pickle', 'wb') as handle:
    pickle.dump((nuc_test_X, nuc_test_Y), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
with open('nuclearity_test_set.pickle', 'rb') as handle:
    (nuc_test_X, nuc_test_Y) = pickle.load(handle)

## Build model

### Build and train the network

In [1239]:
nuclearity_model = Sequential()
nuclearity_model.add(Dense(256, input_dim=nuc_train_X.shape[1], activation='relu'))
nuclearity_model.add(Dropout(0.5))
nuclearity_model.add(Dense(128, activation='relu'))
nuclearity_model.add(Dropout(0.5))
nuclearity_model.add(Dense(64, activation='relu'))
nuclearity_model.add(Dropout(0.5))
nuclearity_model.add(Dense(nuc_train_Y.shape[1], activation='softmax'))
nuclearity_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [1326]:
nuclearity_model.fit(nuc_train_X, nuc_train_Y, batch_size=128, verbose=0, epochs=10)
nuclearity_model.save("nuclearity_model.h5")

### Or just load it

In [45]:
nuclearity_model = load_model("nuclearity_model.h5")

### And evaluate it

In [46]:
score = nuclearity_model.evaluate(nuc_train_X, nuc_train_Y, verbose=0)
print(score)

[0.50873765834370721, 0.95097068899885806]


In [47]:
score = nuclearity_model.evaluate(nuc_test_X, nuc_test_Y, verbose=0)
print(score)

[0.62963520883748714, 0.91784338911323404]


# 7. Tree construction

In [48]:
sent1 = ['Spencer', 'J', '.', 'Volk', ',', 'president', 'and', 'chief', 'operating', 'officer', 'of', 'this', 'consumer', 'and', 'industrial', 'products', 'company', ',', 'was', 'elected', 'a', 'director', '.']
sent2 = ['Mr', '.', 'Volk', ',', '55', 'years', 'old', ',', 'succeeds', 'Duncan', 'Dwight', ',']
sent3 = ['who', 'retired', 'in', 'September', '.']

In [49]:
idx2relation = { index : relation for relation, index in relation2idx.items() }

In [50]:
def merge_once(tree_list):
    best_connection = 0
    best_connection_index = -1
    best_vector = None
    for i, (lhs, rhs) in enumerate(zip(tree_list, tree_list[1:])):
        vector = get_vector(lhs, rhs)
        if vector is None:
            continue
        connection = connection_model.predict(np.expand_dims(vector, axis=0))[0][0]
        #print(connection)
        if connection > best_connection:
            best_connection = connection
            best_connection_index = i
            best_vector = vector

    if best_vector is None:
        return tree_list

    lhs = tree_list[best_connection_index]
    rhs = tree_list[best_connection_index + 1]

    relation = relation_model.predict_classes(np.expand_dims(best_vector, axis=0))[0]
    relation = idx2relation[relation]
    #print(relation)

    nuclearity = nuclearity_model.predict(np.expand_dims(best_vector, axis=0))[0]
    nuclearity = np.round(nuclearity).astype(np.int)
    #print(nuclearity)

    nuclei = []
    satellite = None
    if nuclearity[0] == 0:
        satellite = lhs
    else:
        nuclei.append(lhs)
    if nuclearity[1] == 0:
        satellite = rhs
    else:
        nuclei.append(rhs)

    new_tree = RSTTree.from_trees(relation, nuclei, satellite)
    new_tree.construct_text()
    new_tree_list = tree_list[:best_connection_index]
    new_tree_list.append(new_tree)
    new_tree_list += tree_list[best_connection_index + 2:]
    return new_tree_list

In [51]:
tree_list = [RSTTree.from_text(sent1, 1, 1),
             RSTTree.from_text(sent2, 2, 2),
             RSTTree.from_text(sent3, 3, 3)]

In [52]:
def merge(tree_list):
    prev_length = len(tree_list)
    while True:
        tree_list = merge_once(tree_list)
        if len(tree_list) == prev_length:
            return tree_list
        prev_length = len(tree_list)

In [53]:
result = merge(tree_list)
print(result[0].output_lisp())

/Users/vpraid/anaconda3/envs/advanced-nlp/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


( root (span 1 3) 
  ( nucleus (span 1 2) (rel2par span)   
    ( nucleus (leaf 1) (rel2par span) (text Spencer J . Volk , president and chief operating officer of this consumer and industrial products company , was elected a director .) )  
    ( satellite (leaf 2) (rel2par elaboration-additional) (text Mr . Volk , 55 years old , succeeds Duncan Dwight ,) )  
)
  ( satellite (leaf 3) (rel2par elaboration-additional) (text who retired in September .) )
)
